In [1]:
# Training: 2020-02-15 to 2020-05-11
# Test: 2020-05-12 to 2020-06-10
# With SMA, the training window is a rolling window

# Countries to work with:
# - Canada
# - Japan
# - Italy
# - UK
# - Germany 
# - France

In [2]:
%python
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [3]:
display(spark.sql("SELECT * FROM g6_confirmed_cases_v2"))

Date,canada_cases,japan_cases,italy_cases,uk_cases,germany_cases,france_cases
2020-02-15T00:00:00.000+0000,0,14,0,0,0,1
2020-02-16T00:00:00.000+0000,0,16,0,0,0,0
2020-02-17T00:00:00.000+0000,1,7,0,0,0,0
2020-02-18T00:00:00.000+0000,0,8,0,0,0,0
2020-02-19T00:00:00.000+0000,0,10,0,0,0,0
2020-02-20T00:00:00.000+0000,0,10,0,0,0,0
2020-02-21T00:00:00.000+0000,1,11,17,0,0,0
2020-02-22T00:00:00.000+0000,0,17,42,0,0,0
2020-02-23T00:00:00.000+0000,0,25,93,0,0,0
2020-02-24T00:00:00.000+0000,1,12,74,4,0,0


In [4]:
%python

# This can be used for the g6_confirmed_cases dataset
def sma(Days, Lag, Country, Prediction_Start):
  Country_Query = spark.sql("SELECT ROW_NUMBER() OVER(ORDER BY Date ASC) AS Row, Date, " + Country + ", lag(MA10," + str(Lag) + ") over(ORDER BY Date) AS Prediction FROM (SELECT " + Country + ", Date, avg(" + Country + ") over(ORDER BY Date ASC ROWS " + str(Days-1) + " PRECEDING) AS MA10 FROM g6_confirmed_cases_v2)")
  Country_Query_Prediction = Country_Query.select('Row', 'Date', Country, 'Prediction').where(Country_Query['Row'] > Prediction_Start)
  #display(Country_Query_Prediction)
  return Country_Query_Prediction

# This can be used for the g6_confirmed_cases dataset for plotting
def sma_plot(Days, Lag, Country, Prediction_Start):
  Country_Query = spark.sql("SELECT ROW_NUMBER() OVER(ORDER BY Date ASC) AS Row, Date, " + Country + ", lag(MA10," + str(Lag) + ") over(ORDER BY Date) AS Prediction FROM (SELECT " + Country + ", Date, avg(" + Country + ") over(ORDER BY Date ASC ROWS " + str(Days-1) + " PRECEDING) AS MA10 FROM g6_confirmed_cases_v2)")
  Country_Query_Prediction = Country_Query.select('Row', 'Date', Country, 'Prediction').where(Country_Query['Row'] > Prediction_Start)
  display(Country_Query_Prediction)
  return Country_Query_Prediction

In [5]:
canada_1_1 = sma_plot(1, 1, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1173.0
89,2020-05-13T00:00:00.000+0000,1149,1155.0
90,2020-05-14T00:00:00.000+0000,1213,1149.0
91,2020-05-15T00:00:00.000+0000,1178,1213.0
92,2020-05-16T00:00:00.000+0000,1247,1178.0
93,2020-05-17T00:00:00.000+0000,1126,1247.0
94,2020-05-18T00:00:00.000+0000,1079,1126.0
95,2020-05-19T00:00:00.000+0000,1082,1079.0
96,2020-05-20T00:00:00.000+0000,1082,1082.0
97,2020-05-21T00:00:00.000+0000,1167,1082.0


In [6]:
canada_7_1 = sma_plot(7, 1, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1329.5714285714287
89,2020-05-13T00:00:00.000+0000,1149,1314.857142857143
90,2020-05-14T00:00:00.000+0000,1213,1267.7142857142858
91,2020-05-15T00:00:00.000+0000,1178,1225.7142857142858
92,2020-05-16T00:00:00.000+0000,1247,1183.5714285714287
93,2020-05-17T00:00:00.000+0000,1126,1184.0
94,2020-05-18T00:00:00.000+0000,1079,1177.2857142857142
95,2020-05-19T00:00:00.000+0000,1082,1163.857142857143
96,2020-05-20T00:00:00.000+0000,1082,1153.4285714285713
97,2020-05-21T00:00:00.000+0000,1167,1143.857142857143


In [7]:
canada_21_1 = sma_plot(21, 1, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1562.6666666666667
89,2020-05-13T00:00:00.000+0000,1149,1534.6190476190477
90,2020-05-14T00:00:00.000+0000,1213,1492.904761904762
91,2020-05-15T00:00:00.000+0000,1178,1472.7619047619048
92,2020-05-16T00:00:00.000+0000,1247,1451.7142857142858
93,2020-05-17T00:00:00.000+0000,1126,1441.952380952381
94,2020-05-18T00:00:00.000+0000,1079,1416.4285714285713
95,2020-05-19T00:00:00.000+0000,1082,1392.4285714285713
96,2020-05-20T00:00:00.000+0000,1082,1370.904761904762
97,2020-05-21T00:00:00.000+0000,1167,1340.7619047619048


In [8]:
canada_1_7 = sma_plot(1, 7, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1258.0
89,2020-05-13T00:00:00.000+0000,1149,1479.0
90,2020-05-14T00:00:00.000+0000,1213,1507.0
91,2020-05-15T00:00:00.000+0000,1178,1473.0
92,2020-05-16T00:00:00.000+0000,1247,1244.0
93,2020-05-17T00:00:00.000+0000,1126,1173.0
94,2020-05-18T00:00:00.000+0000,1079,1173.0
95,2020-05-19T00:00:00.000+0000,1082,1155.0
96,2020-05-20T00:00:00.000+0000,1082,1149.0
97,2020-05-21T00:00:00.000+0000,1167,1213.0


In [9]:
canada_7_7 = sma_plot(7, 7, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1644.4285714285713
89,2020-05-13T00:00:00.000+0000,1149,1610.7142857142858
90,2020-05-14T00:00:00.000+0000,1213,1598.5714285714287
91,2020-05-15T00:00:00.000+0000,1178,1539.7142857142858
92,2020-05-16T00:00:00.000+0000,1247,1491.2857142857142
93,2020-05-17T00:00:00.000+0000,1126,1369.5714285714287
94,2020-05-18T00:00:00.000+0000,1079,1329.5714285714287
95,2020-05-19T00:00:00.000+0000,1082,1314.857142857143
96,2020-05-20T00:00:00.000+0000,1082,1267.7142857142858
97,2020-05-21T00:00:00.000+0000,1167,1225.7142857142858


In [10]:
canada_21_7 = sma_plot(21, 7, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1657.857142857143
89,2020-05-13T00:00:00.000+0000,1149,1672.3809523809523
90,2020-05-14T00:00:00.000+0000,1213,1647.7142857142858
91,2020-05-15T00:00:00.000+0000,1178,1622.3809523809523
92,2020-05-16T00:00:00.000+0000,1247,1608.1904761904761
93,2020-05-17T00:00:00.000+0000,1126,1603.2380952380952
94,2020-05-18T00:00:00.000+0000,1079,1562.6666666666667
95,2020-05-19T00:00:00.000+0000,1082,1534.6190476190477
96,2020-05-20T00:00:00.000+0000,1082,1492.904761904762
97,2020-05-21T00:00:00.000+0000,1167,1472.7619047619048


In [11]:
canada_1_21 = sma_plot(1, 21, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1744.0
89,2020-05-13T00:00:00.000+0000,1149,2025.0
90,2020-05-14T00:00:00.000+0000,1213,1636.0
91,2020-05-15T00:00:00.000+0000,1178,1620.0
92,2020-05-16T00:00:00.000+0000,1247,1452.0
93,2020-05-17T00:00:00.000+0000,1126,1662.0
94,2020-05-18T00:00:00.000+0000,1079,1583.0
95,2020-05-19T00:00:00.000+0000,1082,1534.0
96,2020-05-20T00:00:00.000+0000,1082,1715.0
97,2020-05-21T00:00:00.000+0000,1167,1592.0


In [12]:
canada_7_21 = sma_plot(7, 21, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1684.5714285714287
89,2020-05-13T00:00:00.000+0000,1149,1806.142857142857
90,2020-05-14T00:00:00.000+0000,1213,1750.5714285714287
91,2020-05-15T00:00:00.000+0000,1178,1695.5714285714287
92,2020-05-16T00:00:00.000+0000,1247,1682.7142857142858
93,2020-05-17T00:00:00.000+0000,1126,1737.7142857142858
94,2020-05-18T00:00:00.000+0000,1079,1674.5714285714287
95,2020-05-19T00:00:00.000+0000,1082,1644.5714285714287
96,2020-05-20T00:00:00.000+0000,1082,1600.2857142857142
97,2020-05-21T00:00:00.000+0000,1167,1594.0


In [13]:
canada_21_21 = sma_plot(21, 21, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1407.0
89,2020-05-13T00:00:00.000+0000,1149,1454.2380952380952
90,2020-05-14T00:00:00.000+0000,1213,1450.047619047619
91,2020-05-15T00:00:00.000+0000,1178,1472.2857142857142
92,2020-05-16T00:00:00.000+0000,1247,1515.6666666666667
93,2020-05-17T00:00:00.000+0000,1126,1498.3809523809523
94,2020-05-18T00:00:00.000+0000,1079,1535.3333333333333
95,2020-05-19T00:00:00.000+0000,1082,1546.047619047619
96,2020-05-20T00:00:00.000+0000,1082,1567.2857142857142
97,2020-05-21T00:00:00.000+0000,1167,1571.047619047619


In [14]:
%python

# Equation from -> https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error
def SMAPE(Country_Query_Prediction, Country):
  SMAPE = Country_Query_Prediction.withColumn("SMAPE_num", abs(col("Prediction")-col(Country)))
  SMAPE = SMAPE.withColumn("SMAPE_den", (abs(col("Prediction"))+abs(col(Country)))/1) #previously it was divide by 2 but the group decided this factor was not needed to use the alternative formula for SMAPE.
  SMAPE = SMAPE.withColumn("SMAPE_n", col("SMAPE_num")/col("SMAPE_den"))
  SMAPE_value = SMAPE.groupby().sum("SMAPE_n").collect()[0][0]/SMAPE.count()*100
  return SMAPE_value

def SMAPE_table(Country):
  # Different parameters for moving average
  result_1_1 = sma(1, 1, Country, 87)
  result_7_1 = sma(7, 1, Country, 87)
  result_14_1 = sma(14, 1, Country, 87)
  result_21_1 = sma(21, 1, Country, 87)
  result_1_7 = sma(1, 7, Country, 87)
  result_7_7 = sma(7, 7, Country, 87)
  result_14_7 = sma(14, 7, Country, 87)
  result_21_7 = sma(21, 7, Country, 87)
  result_1_14 = sma(1, 14, Country, 87)
  result_7_14 = sma(7, 14, Country, 87)
  result_14_14 = sma(14, 14, Country, 87)
  result_21_14 = sma(21, 14, Country, 87)
  result_1_21 = sma(1, 21, Country, 87)
  result_7_21 = sma(7, 21, Country, 87)
  result_14_21 = sma(14, 21, Country, 87)
  result_21_21 = sma(21, 21, Country, 87)

  # Creating a dataframe to make a table to summarize the results
  Results = Row("Model", "1 Day", "7 Day", "14 Day", "21 Day")
  results1 = Results("1 Day Prediction", SMAPE(result_1_1, Country), SMAPE(result_7_1, Country), SMAPE(result_14_1, Country), SMAPE(result_21_1, Country))
  results2 = Results("7 Day Prediction", SMAPE(result_1_7, Country), SMAPE(result_7_7, Country), SMAPE(result_14_7, Country), SMAPE(result_21_7, Country))
  results3 = Results("14 Day Prediction", SMAPE(result_1_14, Country), SMAPE(result_7_14, Country), SMAPE(result_14_14, Country), SMAPE(result_21_14, Country))
  results4 = Results("21 Day Prediction", SMAPE(result_1_21, Country), SMAPE(result_7_21, Country), SMAPE(result_14_21, Country), SMAPE(result_21_21, Country))
  df = spark.createDataFrame([results1, results2, results3, results4])
  df = df.withColumn(
    "1 Day", func.round(df["1 Day"], 2)).withColumn(
    "7 Day", func.round(df["7 Day"],2)).withColumn(
    "14 Day", func.round(df["14 Day"],2)).withColumn(
    "21 Day", func.round(df["21 Day"],2))
  print(Country)
  df.show()

In [15]:
#RMSE Functions

def RMSE(Country_Query_Prediction, Country):
  RMSE = Country_Query_Prediction.withColumn("RMSE_num", (col("Prediction")-col(Country))**2)
  RMSE_value = ((RMSE.groupby().sum("RMSE_num").collect()[0][0]/RMSE.count())**0.5)
  return RMSE_value

def RMSE_table(Country):
  # Different parameters for moving average
  result_1_1 = sma(1, 1, Country, 87)
  result_7_1 = sma(7, 1, Country, 87)
  result_14_1 = sma(14, 1, Country, 87)
  result_21_1 = sma(21, 1, Country, 87)
  result_1_7 = sma(1, 7, Country, 87)
  result_7_7 = sma(7, 7, Country, 87)
  result_14_7 = sma(14, 7, Country, 87)
  result_21_7 = sma(21, 7, Country, 87)
  result_1_14 = sma(1, 14, Country, 87)
  result_7_14 = sma(7, 14, Country, 87)
  result_14_14 = sma(14, 14, Country, 87)
  result_21_14 = sma(21, 14, Country, 87)
  result_1_21 = sma(1, 21, Country, 87)
  result_7_21 = sma(7, 21, Country, 87)
  result_14_21 = sma(14, 21, Country, 87)
  result_21_21 = sma(21, 21, Country, 87)

  # Creating a dataframe to make a table to summarize the results
  Results = Row("Model", "1 Day", "7 Day", "14 Day", "21 Day")
  results1 = Results("1 Day Prediction", RMSE(result_1_1, Country), RMSE(result_7_1, Country), RMSE(result_14_1, Country), RMSE(result_21_1, Country))
  results2 = Results("7 Day Prediction", RMSE(result_1_7, Country), RMSE(result_7_7, Country), RMSE(result_14_7, Country), RMSE(result_21_7, Country))
  results3 = Results("14 Day Prediction", RMSE(result_1_14, Country), RMSE(result_7_14, Country), RMSE(result_14_14, Country), RMSE(result_21_14, Country))
  results4 = Results("21 Day Prediction", RMSE(result_1_21, Country), RMSE(result_7_21, Country), RMSE(result_14_21, Country), RMSE(result_21_21, Country))
  df = spark.createDataFrame([results1, results2, results3, results4])
  df = df.withColumn(
    "1 Day", func.round(df["1 Day"], 2)).withColumn(
    "7 Day", func.round(df["7 Day"],2)).withColumn(
    "14 Day", func.round(df["14 Day"],2)).withColumn(
    "21 Day", func.round(df["21 Day"],2))
  print(Country)
  df.show()


In [16]:
# R^2 Functions
# Equation from -> https://en.wikipedia.org/wiki/Coefficient_of_determination
def R2(Country_Query_Prediction, Country):
  R2 = Country_Query_Prediction
  R2_avg = R2.groupby().avg(Country).collect()[0][0]
  R2 = R2.withColumn("R2_TSS", (col(Country)-R2_avg)**2)
  R2 = R2.withColumn("R2_RSS", (col(Country)-col("Prediction"))**2)
  R2_value = (1 - (R2.groupby().sum("R2_RSS").collect()[0][0])/(R2.groupby().sum("R2_TSS").collect()[0][0]))*100 # Multiply by 100 for percentage
  return R2_value

def R2_table(Country):
  # Different parameters for moving average
  result_1_1 = sma(1, 1, Country, 87)
  result_7_1 = sma(7, 1, Country, 87)
  result_14_1 = sma(14, 1, Country, 87)
  result_21_1 = sma(21, 1, Country, 87)
  result_1_7 = sma(1, 7, Country, 87)
  result_7_7 = sma(7, 7, Country, 87)
  result_14_7 = sma(14, 7, Country, 87)
  result_21_7 = sma(21, 7, Country, 87)
  result_1_14 = sma(1, 14, Country, 87)
  result_7_14 = sma(7, 14, Country, 87)
  result_14_14 = sma(14, 14, Country, 87)
  result_21_14 = sma(21, 14, Country, 87)
  result_1_21 = sma(1, 21, Country, 87)
  result_7_21 = sma(7, 21, Country, 87)
  result_14_21 = sma(14, 21, Country, 87)
  result_21_21 = sma(21, 21, Country, 87)

  # Creating a dataframe to make a table to summarize the results
  Results = Row("Model", "1 Day", "7 Day", "14 Day", "21 Day")
  results1 = Results("1 Day Prediction", R2(result_1_1, Country), R2(result_7_1, Country), R2(result_14_1, Country), R2(result_21_1, Country))
  results2 = Results("7 Day Prediction", R2(result_1_7, Country), R2(result_7_7, Country), R2(result_14_7, Country), R2(result_21_7, Country))
  results3 = Results("14 Day Prediction", R2(result_1_14, Country), R2(result_7_14, Country), R2(result_14_14, Country), R2(result_21_14, Country))
  results4 = Results("21 Day Prediction", R2(result_1_21, Country), R2(result_7_21, Country), R2(result_14_21, Country), R2(result_21_21, Country))
  df = spark.createDataFrame([results1, results2, results3, results4])
  df = df.withColumn(
    "1 Day", func.round(df["1 Day"], 2)).withColumn(
    "7 Day", func.round(df["7 Day"],2)).withColumn(
    "14 Day", func.round(df["14 Day"],2)).withColumn(
    "21 Day", func.round(df["21 Day"],2))
  print(Country)
  df.show()


In [17]:
RMSE_table("canada_cases")
RMSE_table("japan_cases")
RMSE_table("italy_cases")
RMSE_table("uk_cases")
RMSE_table("germany_cases")
RMSE_table("france_cases")

canada_cases
+-----------------+------+------+------+------+
 Model| 1 Day| 7 Day|14 Day|21 Day|
+-----------------+------+------+------+------+
 1 Day Prediction| 91.77|127.36|221.28|298.72|
 7 Day Prediction|206.93|298.11|369.06|438.81|
14 Day Prediction|418.52|452.64|516.97|566.65|
21 Day Prediction|547.56|589.47|631.04|651.06|
+-----------------+------+------+------+------+

japan_cases
+-----------------+------+------+------+------+
 Model| 1 Day| 7 Day|14 Day|21 Day|
+-----------------+------+------+------+------+
 1 Day Prediction| 30.15| 25.6| 48.82| 92.11|
 7 Day Prediction| 41.42| 69.48|116.51|163.74|
14 Day Prediction|108.06|163.03|212.01| 255.0|
21 Day Prediction|241.97|265.93|306.55|330.08|
+-----------------+------+------+------+------+

italy_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction| 215.19| 176.37| 282.87| 469.79|
 7 Day Prediction| 274.82| 366.27| 574.95| 833.82|
14 Day Prediction| 608.94| 789.55|1071.56|1361.88|
21 Day Prediction|1140.56|1355.92| 1651.3|1949.26|
+-----------------+-------+-------+-------+-------+

uk_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction| 856.31| 810.18|1032.66|1256.93|
 7 Day Prediction| 1205.5|1259.42|1493.51|1722.99|
14 Day Prediction| 1805.5|1787.24|2007.09|2208.12|
21 Day Prediction|2088.35|2272.84| 2458.0|2539.41|
+-----------------+-------+-------+-------+-------+

germany_cases
+-----------------+------+------+------+-------+
 Model| 1 Day| 7 Day|14 Day| 21 Day|
+-----------------+------+------+------+-------+
 1 Day Prediction|325.27|248.43|265.58| 340.24|
 7 Day Prediction|318.17|287.22|382.75| 538.58|
14 Day Prediction|444.14|494.69|679.86| 916.26|
21 Day Prediction|739.35|876.58|1137.3|1542.67|
+-----------------+------+------+------+-------+

france_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction|1132.97| 986.4|1017.21|1071.27|
 7 Day Prediction|1327.69|1104.86|1158.83|1247.29|
14 Day Prediction|1268.34|1286.96|1391.81|1604.99|
21 Day Prediction|1585.82|1538.83|1812.46|2035.69|
+-----------------+-------+-------+-------+-------+

In [18]:
SMAPE_table("canada_cases")
SMAPE_table("japan_cases")
SMAPE_table("italy_cases")
SMAPE_table("UK_cases")
SMAPE_table("germany_cases")
SMAPE_table("france_cases")

canada_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|21 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction| 4.25| 6.57| 10.96| 14.62|
 7 Day Prediction| 9.7|14.01| 17.39| 20.26|
14 Day Prediction|17.93|20.51| 23.05| 24.66|
21 Day Prediction|23.06|25.33| 26.45| 26.38|
+-----------------+-----+-----+------+------+

japan_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|21 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction|28.01|19.62| 25.97| 33.98|
 7 Day Prediction|25.31|29.06| 37.06| 47.65|
14 Day Prediction|38.04|43.33| 53.84| 63.99|
21 Day Prediction|51.71|61.15| 69.63| 75.0|
+-----------------+-----+-----+------+------+

italy_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|21 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction|15.34|13.51| 20.75| 29.59|
 7 Day Prediction|18.33|25.37| 34.04| 42.65|
14 Day Prediction|33.14|40.89| 48.58| 55.35|
21 Day Prediction|48.39|54.45| 60.18| 64.99|
+-----------------+-----+-----+------+------+

UK_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|21 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction|13.54|12.63| 16.83| 21.11|
 7 Day Prediction|18.57|20.18| 24.12| 28.04|
14 Day Prediction|27.17|27.67| 31.42| 34.04|
21 Day Prediction|31.16|34.47| 36.5| 37.1|
+-----------------+-----+-----+------+------+

germany_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|21 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction|24.97|21.64| 23.02| 26.98|
 7 Day Prediction|25.02|23.77| 29.08| 34.77|
14 Day Prediction|30.28|34.02| 39.56| 46.75|
21 Day Prediction|39.49| 44.6| 51.52| 58.45|
+-----------------+-----+-----+------+------+

france_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|21 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction|43.68|41.37| 42.64| 47.98|
 7 Day Prediction|43.52|45.39| 49.23| 52.72|
14 Day Prediction|44.54|51.24| 53.99| 57.83|
21 Day Prediction|55.08| 55.9| 60.6| 64.02|
+-----------------+-----+-----+------+------+

In [19]:
R2_table("canada_cases")
R2_table("japan_cases")
R2_table("italy_cases")
R2_table("UK_cases")
R2_table("germany_cases")
R2_table("france_cases")

canada_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction| 85.52| 72.11| 15.81| -53.43|
 7 Day Prediction| 26.38| -52.8|-134.19|-231.07|
14 Day Prediction|-201.17|-252.27|-359.52|-452.08|
21 Day Prediction|-415.51|-497.43|-584.68| -628.8|
+-----------------+-------+-------+-------+-------+

japan_cases
+-----------------+--------+---------+--------+---------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+--------+---------+--------+---------+
 1 Day Prediction| -48.65| -7.24| -289.92| -1287.71|
 7 Day Prediction| -180.57| -689.63|-2120.54| -4285.74|
14 Day Prediction|-1810.08| -4247.43|-7252.37|-10536.76|
21 Day Prediction|-9477.02|-11467.68|-15271.5|-17721.89|
+-----------------+--------+---------+--------+---------+

italy_cases
+-----------------+--------+--------+--------+--------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+--------+--------+--------+--------+
 1 Day Prediction| 40.16| 59.81| -3.39| -185.18|
 7 Day Prediction| 2.41| -73.35| -327.14| -798.39|
14 Day Prediction| -379.14| -705.52|-1383.72|-2296.62|
21 Day Prediction|-1580.95|-2275.67|-3423.45|-4809.73|
+-----------------+--------+--------+--------+--------+

UK_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction| 6.71| 16.49| -35.68|-101.01|
 7 Day Prediction| -84.9|-101.81| -183.8|-277.71|
14 Day Prediction|-314.75|-306.41|-412.54|-520.36|
21 Day Prediction|-454.89|-557.25|-668.71|-720.47|
+-----------------+-------+-------+-------+-------+

germany_cases
+-----------------+-------+--------+--------+--------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+-------+--------+--------+--------+
 1 Day Prediction| -69.11| 1.36| -12.74| -85.02|
 7 Day Prediction| -61.8| -31.85| -134.15| -363.62|
14 Day Prediction|-215.28| -291.14| -638.77|-1241.85|
21 Day Prediction| -773.7|-1128.13|-1967.36|-3703.74|
+-----------------+-------+--------+--------+--------+

france_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 21 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction| -44.18| -9.29| -16.22| -28.91|
 7 Day Prediction| -98.0| -37.12| -50.84| -74.74|
14 Day Prediction| -80.69| -86.04|-117.59|-189.35|
21 Day Prediction|-182.48|-165.98|-268.98|-365.48|
+-----------------+-------+-------+-------+-------+